In [17]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-


import numpy as np 
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Get distances clean

In [18]:
import TRACE_module.preprocessing as pp

import TRACE_module.visualisation as vi
from TRACE_module.descriptive_analysis import from_stack_to_number_of_interaction_sequences,from_distance_to_sequences_vector, from_distances_to_sequences_stack,from_seq_to_average_interaction_time,from_seq_to_daily_interactions
from TRACE_module.apriori_spade import stack_to_one_hot_df, get_maximum_connex_graph, apriori_
from TRACE_module.DTGC import create_file_x_y_t as create_file_x_y_t
from TRACE_module.DTGC import create_file_nodeId_label as create_file_nodeId_label
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

import dotenv
import sys
import time
from TRACE_module.env_loading import parent_dir, output_dir

if os.getcwd().endswith("TRACE_module"):
    new_path = os.getcwd()[:-len("TRACE_module")] + "UTILS_module"
elif os.getcwd().endswith("TRACE_IODAA"):
    new_path = os.getcwd() + os.sep + "UTILS_module"
sys.path.insert(0, new_path) # insert path to be able to import file afterwards, allows for imports wherever the script is called from

from UTILS_files import test_import
if test_import() : # Testing the import
    from UTILS_files import get_all_files_within_one_folder # Importing the wanted function
else :
    print("Import failed")



UTILS_files.py imported successfully


In [ ]:
folder= parent_dir
list_files=list()

In [ ]:
list_files = get_all_files_within_one_folder(folder, True, extension=".parquet")

print(list_files)
print(len(list_files))

Time to get all files :  0.0024556669959565625
['/Users/josephallyndree/Dropbox/Joseph/A-PhD/PFR INRAE - Graphes reseaux/20241029 - Buisson/Data_rssi_glob_sensor_time/20241016-20241029_365d_glob_sensor_time_ble.parquet', '/Users/josephallyndree/Dropbox/Joseph/A-PhD/PFR INRAE - Graphes reseaux/20241029 - Buisson/Data_rssi_glob_sensor_time/20241016-20241029_365e_glob_sensor_time_ble.parquet', '/Users/josephallyndree/Dropbox/Joseph/A-PhD/PFR INRAE - Graphes reseaux/20241029 - Buisson/Data_rssi_glob_sensor_time/20241016-20241029_3660_glob_sensor_time_ble.parquet', '/Users/josephallyndree/Dropbox/Joseph/A-PhD/PFR INRAE - Graphes reseaux/20241029 - Buisson/Data_rssi_glob_sensor_time/20241016-20241029_3662_glob_sensor_time_ble.parquet', '/Users/josephallyndree/Dropbox/Joseph/A-PhD/PFR INRAE - Graphes reseaux/20241029 - Buisson/Data_rssi_glob_sensor_time/20241016-20241029_3663_glob_sensor_time_ble.parquet', '/Users/josephallyndree/Dropbox/Joseph/A-PhD/PFR INRAE - Graphes reseaux/20241029 - Bui

In [ ]:
folder_savings =  os.sep.join([output_dir,"savings"])
if not os.path.isdir(folder_savings) :
    os.makedirs(folder_savings)

data=pp.concatenate_df(list_files,smooth_time='20s')
data=pp.remove_captor_(data,["366b"])


KeyboardInterrupt: 

In [7]:
print(data)

       glob_sensor_DateTime accelero_id  RSSI id_sensor
0       2024-10-15 14:16:00        365e -61.0      365d
1       2024-10-15 14:16:20        365e -73.0      365d
2       2024-10-15 14:16:40        365e -78.0      365d
3       2024-10-15 14:17:00        365e -62.0      365d
4       2024-10-15 14:17:20        365e -50.0      365d
...                     ...         ...   ...       ...
436722  2024-10-29 17:29:40        3d0c -59.0      3d0f
436723  2024-10-29 17:30:00        3d0c -61.0      3d0f
436724  2024-10-29 17:30:20        3d0c -59.0      3d0f
436725  2024-10-29 17:30:40        3d0c -57.0      3d0f
436726  2024-10-29 17:31:00        3d0c -55.0      3d0f

[25272458 rows x 4 columns]


In [9]:

data=pp.transform_rssi_to_distance(data)

print(data)

       glob_sensor_DateTime accelero_id  RSSI id_sensor  evaluated_distance
0       2024-10-15 14:16:00        365e -61.0      365d          107.977516
1       2024-10-15 14:16:20        365e -73.0      365d          271.227258
2       2024-10-15 14:16:40        365e -78.0      365d          398.107171
3       2024-10-15 14:17:00        365e -62.0      365d          116.591440
4       2024-10-15 14:17:20        365e -50.0      365d           46.415888
...                     ...         ...   ...       ...                 ...
436722  2024-10-29 17:29:40        3d0c -59.0      3d0f           92.611873
436723  2024-10-29 17:30:00        3d0c -61.0      3d0f          107.977516
436724  2024-10-29 17:30:20        3d0c -59.0      3d0f           92.611873
436725  2024-10-29 17:30:40        3d0c -57.0      3d0f           79.432823
436726  2024-10-29 17:31:00        3d0c -55.0      3d0f           68.129207

[25272458 rows x 5 columns]


In [13]:
list_id=list(pd.unique(data["accelero_id"]))

stack,list_timesteps=pp.create_stack(data,list_id)

# print(stack)
list_timesteps


array(['2024-10-15T14:16:00.000000000', '2024-10-15T14:16:20.000000000',
       '2024-10-15T14:16:40.000000000', ...,
       '2024-10-15T12:45:40.000000000', '2024-10-15T12:46:00.000000000',
       '2024-10-15T12:46:20.000000000'],
      shape=(61654,), dtype='datetime64[ns]')

In [14]:

###BUISSON
start_time = pd.Timestamp('2024-10-18T15:00:00.000000000')
end_time = pd.Timestamp('2024-10-27T06:00:00.000000000')

distances_clean,list_timesteps=pp.crop_start_end_stack(stack=stack,
                                                        list_timesteps = list_timesteps ,
                                                        start = start_time,
                                                        end = end_time)

print(distances_clean)


[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 1 0 0]
  ...
  [0 0 1 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


# Starting HTNE

In [1]:
from TRACE_module.DTGC import convert_HTNE_embFile, TSNE_show

In [7]:
# Loading cow.txt
path_cow_data = "/Users/josephallyndree/Dropbox/Joseph/A-PhD/TRACE_IODAA/cows.txt"

converted_data = convert_HTNE_embFile(path_cow_data, 1000000)

print(converted_data)

[[1.80000000e+01 3.80000000e+01 1.87869028e-04]
 [2.10000000e+01 3.30000000e+01 1.87869028e-04]
 [3.10000000e+01 3.40000000e+01 1.87869028e-04]
 ...
 [1.60000000e+01 2.90000000e+01 3.57326892e-01]
 [1.80000000e+01 3.50000000e+01 3.57326892e-01]
 [2.00000000e+01 3.10000000e+01 3.57326892e-01]]


In [12]:

import torch
from torch.autograd import Variable
from torch.optim import SGD
from torch.utils.data import DataLoader
from torch.nn.functional import softmax
import sys

from torch.utils.data import Dataset

FType = torch.FloatTensor
LType = torch.LongTensor

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print("MPS is available.")

MPS is available.


In [14]:

class HTNEDataSet(Dataset):
    def __init__(self, file_path, neg_size, hist_len, directed=False, transform=None):
        self.neg_size = neg_size
        self.hist_len = hist_len
        self.directed = directed
        self.transform = transform

        # self.max_d_time = -sys.maxint  # Time interval [0, T]
        self.max_d_time = -sys.maxsize
        self.NEG_SAMPLING_POWER = 0.75
        self.neg_table_size = int(1e8)

        self.node2hist = dict()
        self.node_set = set()
        self.degrees = dict()
        with open(file_path, 'r') as infile:
            for line in infile:
                parts = line.split()
                s_node = int(parts[0])  # source node
                t_node = int(parts[1])  # target node
                d_time = float(parts[2])  # time slot, delta t

                self.node_set.update([s_node, t_node])

                if s_node not in self.node2hist:
                    self.node2hist[s_node] = list()
                self.node2hist[s_node].append((t_node, d_time))

                if not directed:
                    if t_node not in self.node2hist:
                        self.node2hist[t_node] = list()
                    self.node2hist[t_node].append((s_node, d_time))

                if d_time > self.max_d_time:
                    self.max_d_time = d_time

                if s_node not in self.degrees:
                    self.degrees[s_node] = 0
                if t_node not in self.degrees:
                    self.degrees[t_node] = 0
                self.degrees[s_node] += 1
                self.degrees[t_node] += 1

        self.node_dim = len(self.node_set)

        self.data_size = 0
        for s in self.node2hist:
            hist = self.node2hist[s]
            hist = sorted(hist, key=lambda x: x[1])
            self.node2hist[s] = hist
            self.data_size += len(self.node2hist[s])

        self.idx2source_id = np.zeros((self.data_size,), dtype=np.int32)
        self.idx2target_id = np.zeros((self.data_size,), dtype=np.int32)
        idx = 0
        for s_node in self.node2hist:
            for t_idx in range(len(self.node2hist[s_node])):
                self.idx2source_id[idx] = s_node
                self.idx2target_id[idx] = t_idx
                idx += 1

        self.neg_table = np.zeros((self.neg_table_size,))
        self.init_neg_table()

    def get_node_dim(self):
        return self.node_dim

    def get_max_d_time(self):
        return self.max_d_time

    def init_neg_table(self):
        tot_sum, cur_sum, por = 0., 0., 0.
        n_id = 0
        print(self.node_dim)
        for k in range(self.node_dim):
            tot_sum += np.power(self.degrees[k], self.NEG_SAMPLING_POWER)
        for k in range(self.neg_table_size):
            if (k + 1.) / self.neg_table_size > por:
                cur_sum += np.power(self.degrees[n_id], self.NEG_SAMPLING_POWER)
                por = cur_sum / tot_sum
                n_id += 1
            self.neg_table[k] = n_id - 1

    def __len__(self):
        return self.data_size

    def __getitem__(self, idx):
        s_node = self.idx2source_id[idx]
        t_idx = self.idx2target_id[idx]
        t_node = self.node2hist[s_node][t_idx][0]
        t_time = self.node2hist[s_node][t_idx][1]

        if t_idx - self.hist_len < 0:
            hist = self.node2hist[s_node][0:t_idx]
        else:
            hist = self.node2hist[s_node][t_idx - self.hist_len:t_idx]
        
        hist_nodes = [h[0] for h in hist]
        hist_times = [h[1] for h in hist]

        np_h_nodes = np.zeros((self.hist_len,))
        np_h_nodes[:len(hist_nodes)] = hist_nodes
        np_h_times = np.zeros((self.hist_len,))
        np_h_times[:len(hist_times)] = hist_times
        np_h_masks = np.zeros((self.hist_len,))
        np_h_masks[:len(hist_nodes)] = 1.

        neg_nodes = self.negative_sampling()
        
        sample = {
            'source_node': s_node,
            'target_node': t_node,
            'target_time': t_time,
            'history_nodes': np_h_nodes,
            'history_times': np_h_times,
            'history_masks': np_h_masks,
            'neg_nodes': neg_nodes,
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

    def negative_sampling(self):
        rand_idx = np.random.randint(0, self.neg_table_size, (self.neg_size,))
        sampled_nodes = self.neg_table[rand_idx]
        return sampled_nodes

In [ ]:
class HTNE_a:
    def __init__(self, file_path, emb_size=128, neg_size=10, hist_len=2, directed=False,
            learning_rate=0.01, batch_size=1000, save_step=50, epoch_num=15):
        self.emb_size = emb_size
        self.neg_size = neg_size
        self.hist_len = hist_len

        self.lr = learning_rate
        self.batch = batch_size
        self.save_step = save_step
        self.epochs = epoch_num

        self.data = HTNEDataSet(file_path, neg_size, hist_len, directed)
        self.node_dim = self.data.get_node_dim()

        if torch.backends.mps.is_available():
            with torch.device("mps"):
                self.node_emb = Variable(torch.from_numpy(np.random.uniform(
                    -1. / np.sqrt(self.node_dim), 1. / np.sqrt(self.node_dim), (self.node_dim, emb_size))).type(
                    FType).to("mps"), requires_grad=True)

                self.delta = Variable((torch.zeros(self.node_dim) + 1.).type(FType).to("mps"), requires_grad=True)

                self.att_param = Variable(torch.diag(torch.from_numpy(np.random.uniform(
                    -1. / np.sqrt(emb_size), 1. / np.sqrt(emb_size), (emb_size,))).type(
                    FType).to("mps")), requires_grad=True)
        else:
            self.node_emb = Variable(torch.from_numpy(np.random.uniform(
                -1. / np.sqrt(self.node_dim), 1. / np.sqrt(self.node_dim), (self.node_dim, emb_size))).type(
                FType), requires_grad=True)

            self.delta = Variable((torch.zeros(self.node_dim) + 1.).type(FType), requires_grad=True)

            self.att_param = Variable(torch.diag(torch.from_numpy(np.random.uniform(
                -1. / np.sqrt(emb_size), 1. / np.sqrt(emb_size), (emb_size,))).type(
                FType)), requires_grad=True)

        self.opt = SGD(lr=learning_rate, params=[self.node_emb, self.att_param, self.delta])
        self.loss = torch.FloatTensor()

    def forward(self, s_nodes, t_nodes, t_times, n_nodes, h_nodes, h_times, h_time_mask):
        batch = s_nodes.size()[0]
        s_node_emb = self.node_emb.index_select(0, Variable(s_nodes.view(-1))).view(batch, -1)
        t_node_emb = self.node_emb.index_select(0, Variable(t_nodes.view(-1))).view(batch, -1)
        h_node_emb = self.node_emb.index_select(0, Variable(h_nodes.view(-1))).view(batch, self.hist_len, -1)

        att = softmax(((s_node_emb.unsqueeze(1) - h_node_emb)**2).sum(dim=2).neg(), dim=1)
        p_mu = ((s_node_emb - t_node_emb)**2).sum(dim=1).neg()
        p_alpha = ((h_node_emb - t_node_emb.unsqueeze(1))**2).sum(dim=2).neg()

        delta = self.delta.index_select(0, Variable(s_nodes.view(-1))).unsqueeze(1)
        d_time = torch.abs(t_times.unsqueeze(1) - h_times)  # (batch, hist_len)
        p_lambda = p_mu + (att * p_alpha * torch.exp(delta * Variable(d_time)) * Variable(h_time_mask)).sum(dim=1)
        
        n_node_emb = self.node_emb.index_select(0, Variable(n_nodes.view(-1))).view(batch, self.neg_size, -1)

        n_mu = ((s_node_emb.unsqueeze(1) - n_node_emb)**2).sum(dim=2).neg()
        n_alpha = ((h_node_emb.unsqueeze(2) - n_node_emb.unsqueeze(1))**2).sum(dim=3).neg()


        n_lambda = n_mu + (att.unsqueeze(2) * n_alpha * (torch.exp(delta * Variable(d_time)).unsqueeze(2)) * (Variable(h_time_mask).unsqueeze(2))).sum(dim=1)
        return p_lambda, n_lambda

    def loss_func(self, s_nodes, t_nodes, t_times, n_nodes, h_nodes, h_times, h_time_mask):
        if torch.backends.mps.is_available():
            with torch.device("mps"):
                p_lambdas, n_lambdas = self.forward(s_nodes, t_nodes, t_times, n_nodes, h_nodes, h_times, h_time_mask)
                loss = -torch.log(p_lambdas.sigmoid() + 1e-6) - torch.log(
                    n_lambdas.neg().sigmoid() + 1e-6).sum(dim=1)

        else:
            p_lambdas, n_lambdas = self.forward(s_nodes, t_nodes, t_times, n_nodes, h_nodes, h_times,
                                                h_time_mask)
            loss = -torch.log(torch.sigmoid(p_lambdas) + 1e-6) - torch.log(
                torch.sigmoid(torch.neg(n_lambdas)) + 1e-6).sum(dim=1)
        return loss

    def update(self, s_nodes, t_nodes, t_times, n_nodes, h_nodes, h_times, h_time_mask):
        if torch.backends.mps.is_available():
            with torch.device("mps"):
                self.opt.zero_grad()
                loss = self.loss_func(s_nodes, t_nodes, t_times, n_nodes, h_nodes, h_times, h_time_mask)
                loss = loss.sum()
                self.loss += loss.data
                loss.backward()
                self.opt.step()
        else:
            self.opt.zero_grad()
            loss = self.loss_func(s_nodes, t_nodes, t_times, n_nodes, h_nodes, h_times, h_time_mask)
            loss = loss.sum()
            self.loss += loss.data
            loss.backward()
            self.opt.step()

    def train(self):
        for epoch in range(self.epochs):
            self.loss = 0.0
            loader = DataLoader(self.data, batch_size=self.batch,
                                shuffle=True, num_workers=5)
            # if epoch % self.save_step == 0 and epoch != 0:
            #     #torch.save(self, './model/dnrl-dblp-%d.bin' % epoch)
            #     self.save_node_embeddings('./emb/cows_htne_attn_%d.emb' % (epoch))

            for i_batch, sample_batched in enumerate(loader):
                if i_batch % 100 == 0 and i_batch != 0:
                    sys.stdout.write('\r' + str(i_batch * self.batch) + '\tloss: ' + str(
                        self.loss.cpu().numpy() / (self.batch * i_batch)) + '\tdelta:' + str(
                        self.delta.mean().cpu().data.numpy()))
                    sys.stdout.flush()

                if torch.backends.mps.is_available():
                    with torch.device("mps"):
                        self.update(sample_batched['source_node'].type(LType).to("mps"),
                                    sample_batched['target_node'].type(LType).to("mps"),
                                    sample_batched['target_time'].type(FType).to("mps"),
                                    sample_batched['neg_nodes'].type(LType).to("mps"),
                                    sample_batched['history_nodes'].type(LType).to("mps"),
                                    sample_batched['history_times'].type(FType).to("mps"),
                                    sample_batched['history_masks'].type(FType).to("mps"))
                else:
                    self.update(sample_batched['source_node'].type(LType),
                                sample_batched['target_node'].type(LType),
                                sample_batched['target_time'].type(FType),
                                sample_batched['neg_nodes'].type(LType),
                                sample_batched['history_nodes'].type(LType),
                                sample_batched['history_times'].type(FType),
                                sample_batched['history_masks'].type(FType))

            sys.stdout.write('\repoch ' + str(epoch) + ': avg loss = ' +
                                str(self.loss.cpu().numpy() / len(self.data)) + '\n')
            sys.stdout.flush()

            self.save_node_embeddings('./emb/cows_htne_attn_%d.emb' % (epoch))

    def save_node_embeddings(self, path):
        if torch.backends.mps.is_available():
            embeddings = self.node_emb.cpu().data.numpy()
        else:
            embeddings = self.node_emb.data.numpy()
        writer = open(path, 'w')
        writer.write('%d %d\n' % (self.node_dim, self.emb_size))
        for n_idx in range(self.node_dim):
            writer.write(' '.join(str(d) for d in embeddings[n_idx]) + '\n')

        writer.close()



In [22]:
model = HTNE_a(
    file_path=path_cow_data,
    emb_size=128,
    neg_size=10,
    hist_len=2,
    directed=False,
    epoch_num=35
)

48


In [23]:
model.train()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
    from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=83, pipe_handle=97)
                                                  ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/josephallyndree/miniconda3/envs/PFR_graph/lib/python3.13/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/josephallyndree/miniconda3/envs/PFR_graph/lib/python3.13/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'HTNEDataSet' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


KeyboardInterrupt: 